#install hazm

In [ ]:
pip install hazm

  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.6/60.6 kB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 892.6/892.6 kB 16.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.3/17.3 MB 30.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.4/4.4 MB 29.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 26.7/26.7 MB 40.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 34.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 292.1/292.1 kB 20.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 38.6/38.6 MB 11.8 MB/s eta 0:00:00
  Created wheel for flashtext: filename=flashtext-2.7-py2.py3-none-any.whl size=9300 sha256=cde837c3896c50bf05cda1c71492f69e1a7abc175eac0dcfb844daed058a57ef
  Stored in directory: /root/.cache/pip/wheels/49/20/47/f03dfa8a7239c54cbc44ff7389eefbf888d2c1873edaaec888
Successfully built flashtext
  Attempting uninstall: numpy
   

In [ ]:
!pip install parsivar

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.0/18.0 MB 15.9 MB/s eta 0:00:00


In [ ]:
!pip install llama-cpp-python

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.6/50.6 MB 11.8 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.5/45.5 kB 3.4 MB/s eta 0:00:00
  Created wheel for llama-cpp-python: filename=llama_cpp_python-0.3.15-cp311-cp311-linux_x86_64.whl size=4397851 sha256=5b80aaa38b784c952f8be7ada8b1eb2d9b014e20cf0e929ff2a2675343be06b1
  Stored in directory: /root/.cache/pip/wheels/34/62/22/63039c8c4c8da7ff68e81b8357e64c199faff9df9a2b5e5e3b
Successfully built llama-cpp-python


#imports

In [ ]:
from hazm import Normalizer, word_tokenize
import pandas as pd
import re

In [ ]:
from parsivar import Normalizer, SpellCheck

In [ ]:
import torch
import transformers
from transformers import AutoTokenizer, AutoModelForCausalLM
from huggingface_hub import login
from huggingface_hub import hf_hub_download
from llama_cpp import Llama
import time
import csv

#Texts

In [ ]:
# Raw Farsi sentences (you can add your 20 samples here)
texts = [
    "مي خواهم كتاب را بخوانم",
    "سلام دكتر. فشار خونم خيلي ًبالاست !",
    "دارو را   روزي سه بر بخور",
    "از ديروز سر درد دارم 不客气 و هيچ دارويي نخوردم",
    "چك كردم ولي قند خون نرمال بود",
    "سلام دکتر 😊 دارو رو روزی 3 بار @ صبح‌ها بخور، اوکی؟ ❤️ #ممنون",
]

In [ ]:
the_texts = [
    "برای درمان تب و بدن‌درد،  باید هر 6 ساعت یک عد قرص استامینووفن 500 مصرف کند و هر 12 ساعت یک کپسول آموکسی‌یلین 500 بعد از غذا مصرف شود.خانم کرمی",
    "آقای صادقی به دلیل باید روزی یک عدد قرص آسیترومایسین 250 به مدت سه روز همراه صبحانه مصرف کند سینوسیت حاد و قطره ونیل‌افرین را هر 8 ساعت در هر سوراخ بینی بچکاند.",
    "خانم عسگری به دلیل آلرسی فصلی باید روزی دو عدد قرص سیتریسین همراه غذا مصرف کند و شب‌ها  شربت دیفن‌هیدراین قبل از خواب یک قاشق میل نماید.",
    "کودک هفت‌ساله‌ای که دچار عفونت گلو شده است باید هر 8 ساعت 5 شربت فالکسین مصرف کند سی‌سی و هر 6 ساعت یک قاشق شربت استامینوفن برای تب تجویزه شده است.",
    "آقای حسام برای زفلاکس معده باید شب‌هاقرص واموتیدین 40 قبل از خواب  یک عدد و همراه ناهار یک قرص رانیتیدین مصرف کند.",
    "جهت درمان سرماخوردگی، خانم آقاجانی باید روزی یه بار لوراتادین قرص پس از غذا مصرف کند و شب‌ها قبل از خواب یک قاشق شربت دیفندرامین نیز میل نماید.",
    "خانم فرمند با تشخیص برونخیت باید روزی یک عدد قرص آزیترومایسین با معده حالی و هر 80 ساعت یک عدد استامینوفن 500 مصرف نماید قرص .",
    "روزی دو باربرای درد مواصل، آقای تقوی باید قرص ایبوپروفن 40 را هر 8 ساعت بعد از غذا و نابروکسن 250 را  همراه یک لیوان آب میل کند.",
    "کودک ده‌ساله‌ای که کم‌خونی دارد باید روزی یک قرص آن همراه صبحانه و شربت آسکوربیک اسید نیز روزی دو بار برای جذب بهتر .",
    "خانم مرادی که دچار حساسیت پوستی است باید روزی یک عدد قرص لوراتادین صبح‌ها و شب‌ها پماد هیدروکورنیزون را روی نواحی آسیب‌ بالد.",
    "برای درمان عفونت مجازی ادراری، آقای خلیلی باید هر 112 ساعت یک کپسول سفالسین 500 و روزی سه بار قرص مترونیداول همراه غذا .",
    "خانم داوری که با اسال شدید و التهاب  باید روزی دو بار قرص لوپرامید و هر روز صبح یک عدد قرص دگزامتازون . مصرف مایات فراموش شود.",
    "کودک سه‌ساله با گلودرد ویروسی باید هر 6 ساعت یک قاشق شربت استامینوفن و هر 12 ساعت یک قاشق شربت هیستامین بعد از غذا .",
    "آقای رفیعی با آلرژی تنفسی باید روزی یک عدد قرص فکسوونادین با شکم خالی و شب‌ها قبل از خواب شربت دیفن‌هیدرامین .",
    "خانم اجباری که دچار کمبود ویتامین C شده است باید شربت آسکوربیک سید را روزی دو بار بعد از غذا و قرص ژینک را همراه وعده شام مصرف کند.",
    "برای بهبود خواب، آقای شایان باید هر شب قبل از خواب یک عدد قرص ظولپیدم 10 مصرف کند و درصورت اضطراب، صبح‌ها یک قرص الپرازولان با معده خالی میل نماید.",
    "کودک هشت‌ساله با اورتیت میانی باید هر 8 ساعت 5 سی‌سی شربت سفیکسلیم و شب‌ها قبل از خواب یک قاشق شربت دیفن‌هیدرامین  شود.",
    "خانم نظری برای رفلاکس باید صبح‌ها ناشتا یک عدد قرص امپرازول و شب‌ها بعد از شام یک قرص فاموتیدین .",
    "خانم نادری با سردرد مرمن باید روزی هر 66 ساعت یک عدد قرص استامینوفن 5000 مصرف کند  در صورت شدت درد، پماد موضعی منتول روی شقیشقه‌ها.",
    " برای درمان عفونت شدید، آقای پرویزی باید روزی یک عدد قرص آسیترومایسین و هر 12 ساعت یک عدد آمپول چنتامایسین تزریقی طبق دستور زشک بزند."
  ]

#cleaning

In [ ]:
def clean_symbols_and_emojis(text):

    # Define allowed punctuation: Persian and some general
    allowed_punct = "،.؟؛:!"

    # Build regex pattern:
    # Pattern to match any character NOT in:
    # - Persian Unicode range \u0600-\u06FF
    # - English letters a-zA-Z
    # - Digits 0-9 and Persian digits \u06F0-\u06F9
    # - Half-space \u200c
    # - Whitespace \s
    # - Allowed punctuation
    pattern = fr"[^a-zA-Z0-9\u06F0-\u06F9\u0600-\u06FF\u200c\s{re.escape(allowed_punct)}]"

    # Remove disallowed characters (emojis, symbols, etc.)
    cleaned = re.sub(pattern, "", text)

    return cleaned


#Normalizing

In [ ]:
# Initialize hazm normalizer
normalizer = Normalizer()

def normalize_farsi_sentences(sentences):
    """
    Normalize a list of Farsi sentences using hazm.
    """
    normalized = []
    for sentence in sentences:
        cleaned = normalizer.normalize(sentence)
        normalized.append(cleaned)
    return normalized


#applying cleaning and normalizing

In [ ]:
# Clean + Normalize pipeline
cleaned_texts = [clean_symbols_and_emojis(t) for t in the_texts]
normalized_texts = [normalizer.normalize(t) for t in cleaned_texts]

# Show original, cleaned, and normalized
for i, (orig, clean, norm) in enumerate(zip(the_texts, cleaned_texts, normalized_texts), 1):
    print(f"{i}. Original   : {orig}")
    print(f"   Cleaned    : {clean}")
    print(f"   Normalized : {norm}")
    print("-" * 50)

1. Original   : برای درمان تب و بدن‌درد،  باید هر 6 ساعت یک عد قرص استامینووفن 500 مصرف کند و هر 12 ساعت یک کپسول آموکسی‌یلین 500 بعد از غذا مصرف شود.خانم کرمی
   Cleaned    : برای درمان تب و بدن‌درد،  باید هر 6 ساعت یک عد قرص استامینووفن 500 مصرف کند و هر 12 ساعت یک کپسول آموکسی‌یلین 500 بعد از غذا مصرف شود.خانم کرمی
   Normalized : برای درمان تب و بدن‌درد ، باید هر 6 ساعت یک عد قرص استامینووفن 500 مصرف کند و هر 12 ساعت یک کپسول آموکسی‌یلین 500 بعد از غذا مصرف شود . خانم کرمی
--------------------------------------------------
2. Original   : آقای صادقی به دلیل باید روزی یک عدد قرص آسیترومایسین 250 به مدت سه روز همراه صبحانه مصرف کند سینوسیت حاد و قطره ونیل‌افرین را هر 8 ساعت در هر سوراخ بینی بچکاند.
   Cleaned    : آقای صادقی به دلیل باید روزی یک عدد قرص آسیترومایسین 250 به مدت سه روز همراه صبحانه مصرف کند سینوسیت حاد و قطره ونیل‌افرین را هر 8 ساعت در هر سوراخ بینی بچکاند.
   Normalized : آقای صادقی به دلیل باید روزی یک عدد قرص آسیترومایسین 250 به مدت سه روز همراه صبحانه مصرف کند سینو

#creat a csv

In [ ]:
# Create DataFrame
df = pd.DataFrame({
    "Original": the_texts,
    "Cleaned": cleaned_texts,
    "Normalized": normalized_texts
})
df.to_csv("farsi_cleaned_normalized.csv", index=False, encoding="utf-8-sig")
print("✅ Exported to farsi_cleaned_normalized.csv")

✅ Exported to farsi_cleaned_normalized.csv


# loading and testing model

In [1]:
import re
import torch
import pandas as pd
from typing import List, Tuple
from transformers import AutoTokenizer, AutoModelForMaskedLM

# -------- Model & device ----------
device = "cuda" if torch.cuda.is_available() else "cpu"
model_name = "xlm-roberta-base"

tok = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForMaskedLM.from_pretrained(model_name).to(device)
model.eval()

MASK = tok.mask_token  # for XLM-R this is "<mask>"

# حروف فارسی/عربی (برای جلوگیری از پیشنهادهای غیرفارسی)
PERSIAN_ARABIC_CHARS = r"\u0600-\u06FF\u0750-\u077F\u08A0-\u08FF"
persian_word_re = re.compile(rf"^[{PERSIAN_ARABIC_CHARS}]+(?:[{PERSIAN_ARABIC_CHARS}‌\-]+)*$")  # شامل نیم‌فاصله

def is_persian_word(w: str) -> bool:
    w = w.strip(".,!?؛،:()[]{}«»\"'…")
    if not w:
        return False
    return bool(persian_word_re.match(w))

def split_sentences(text: str) -> List[str]:
    # تفکیک سادهٔ جملات (می‌توانید بعداً NLTK/Parsivar اضافه کنید)
    parts = re.split(r"([\.!\?؟…]+)\s*", text)
    # parts مثل [جمله, نشانه‌گذاری, فاصله, جمله, نشانه‌گذاری, ...]
    sents = []
    buf = ""
    for p in parts:
        if p is None:
            continue
        if re.fullmatch(r"[\.!\?؟…]+", p):
            buf += p
            sents.append(buf.strip())
            buf = ""
        else:
            if p:
                if buf and not buf.endswith(" "):
                    buf += " "
                buf += p.strip()
    if buf.strip():
        sents.append(buf.strip())
    return sents

@torch.no_grad()
def score_mask(sentence_with_mask: str, masked_index: int, original_token_id: int, top_k: int = 10):
    """
    لاگیت‌های پیش‌بینی در موقعیت ماسک را می‌گیرد و:
      - نمرهٔ توکنِ اصلی را برمی‌گرداند
      - بهترین جایگزین‌های مدل را (توکن، نمره، توکن‌آی‌دی)
    """
    enc = tok(sentence_with_mask, return_tensors="pt")
    # اگر جمله خیلی بلند باشد، کوتاه می‌کنیم (XLM-R حداکثر 512)
    for k in enc:
        enc[k] = enc[k][:, :512]
    enc = {k: v.to(device) for k, v in enc.items()}

    out = model(**enc).logits  # [1, seq_len, vocab]
    # پیدا کردن ایندکس <mask> بعد از توکن‌سازی
    mask_token_id = tok.mask_token_id
    mask_positions = (enc["input_ids"][0] == mask_token_id).nonzero(as_tuple=False).squeeze(-1)
    if mask_positions.numel() == 0:
        return None, []
    # اگر چند ماسک بود، فرض می‌کنیم اولی همان است
    mp = mask_positions[0].item()
    logits = out[0, mp]  # [vocab]
    probs = torch.softmax(logits, dim=-1)

    orig_score = probs[original_token_id].item()
    top_scores, top_ids = torch.topk(probs, k=top_k)
    top = [(tok.convert_ids_to_tokens(int(i)), float(s), int(i)) for s, i in zip(top_scores, top_ids)]
    return float(orig_score), top

def rebuild_sentence(tokens: List[str]) -> str:
    # بازسازی جمله از لیست توکن‌ها (فاصله‌گذاریِ ساده)
    sent = " ".join(tokens)
    # تمیزی‌های جزئی: فاصله قبل از علائم فارسی/لاتین
    sent = re.sub(r"\s+([،؛,.!?؟…])", r"\1", sent)
    sent = re.sub(r"«\s+", "«", sent)
    sent = re.sub(r"\s+»", "»", sent)
    return sent

def candidate_is_ok(word: str, cand_token: str) -> bool:
    # cand_token مثل Ġword یا ▁word در برخی مدل‌هاست؛ برای XLM-R معمولاً بدون پیشوند است
    cand = tok.convert_tokens_to_string([cand_token]).strip()
    if not cand:
        return False
    if cand == word:
        return False
    # فقط پیشنهادهای فارسی/عربی را بپذیریم (برای متن فارسی)
    if not is_persian_word(cand):
        return False
    # از کاراکترهای عجیب جلوگیری کنیم
    if any(ch.isdigit() for ch in cand):
        return False
    return True

def correct_sentence_with_xlmr(sentence: str,
                               top_k: int = 8,
                               min_gain: float = 0.30,
                               skip_short: int = 2) -> str:
    """
    min_gain: حداقل بهبود احتمال نسبت به کلمهٔ اصلی تا جایگزینی انجام شود (0.30 = 30%)
    skip_short: کلمات با طول کمتر از این مقدار بررسی نمی‌شوند
    """
    if not sentence or sentence.strip() == "":
        return sentence

    tokens = sentence.split()  # تفکیک ساده با فاصله
    changed = False

    for i, w in enumerate(tokens):
        # از کلمات خیلی کوتاه یا غیرفارسی بگذریم (نویز)
        pure_w = w.strip(".,!?؛،:()[]{}«»\"'…")
        if len(pure_w) < skip_short or not is_persian_word(pure_w):
            continue

        # اگر کلمه به یک زیرتوکن تبدیل نشود، باز هم مشکلی نیست؛
        # چون ما جمله را با یک <mask> می‌سازیم و احتمال همان توکنِ اصلی را می‌سنجیم.
        # جملهٔ ماسک‌شده:
        masked_tokens = tokens.copy()
        masked_tokens[i] = MASK
        masked_sentence = rebuild_sentence(masked_tokens)

        # آی‌دی توکن اصلی (to string -> tokenize -> id اولین بخش)
        # توجه: اگر کلمه به چند زیرتوکن می‌شکند، ما فقط اولین زیرتوکن را به‌عنوان «نماینده» می‌سنجیم.
        orig_ids = tok.encode(pure_w, add_special_tokens=False)
        if not orig_ids:
            continue
        original_subtoken_id = orig_ids[0]

        orig_score, top = score_mask(masked_sentence, i, original_subtoken_id, top_k=top_k)
        if orig_score is None:
            continue

        # بهترین کاندید متفاوت و مناسب
        best = None
        for cand_tok, score, tid in top:
            if candidate_is_ok(pure_w, cand_tok):
                best = (cand_tok, score, tid)
                break

        if best is None:
            continue

        cand_tok, cand_score, _ = best
        # اگر بهبود به‌قدر کافی باشد، جایگزین کن
        if cand_score >= orig_score * (1.0 + min_gain):
            cand_word = tok.convert_tokens_to_string([cand_tok]).strip()
            # حفظ علائم پیرامونی
            prefix = w[:len(w) - len(w.lstrip("«({[\"'"))]
            suffix = w[len(w.rstrip(".,!?؛،:)}]»\"'…")):]
            tokens[i] = f"{prefix}{cand_word}{suffix}"
            changed = True

    return rebuild_sentence(tokens) if changed else sentence

def correct_text(text: str) -> str:
    sents = split_sentences(text)
    fixed = [correct_sentence_with_xlmr(s) for s in sents]
    return " ".join(fixed).strip()

# -------- CSV I/O ----------
in_csv = "farsi_cleaned_normalized.csv"
in_col = "Normalized"
out_col = "xlmr_corrected_on_normalized"
out_csv = "farsi_xlmr_corrected_on_normalized.csv"

df = pd.read_csv(in_csv)
texts = df[in_col].fillna("").astype(str).tolist()

outputs = []
for t in texts:
    outputs.append(correct_text(t))

df[out_col] = outputs
df.to_csv(out_csv, index=False, encoding="utf-8-sig")
print(f"✅ Saved to {out_csv}")


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/615 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.10M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.12G [00:00<?, ?B/s]

Some weights of the model checkpoint at xlm-roberta-base were not used when initializing XLMRobertaForMaskedLM: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing XLMRobertaForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing XLMRobertaForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


✅ Saved to farsi_xlmr_corrected_on_normalized.csv
